In [1]:
# 모듈, 경로 설정
import pandas as pd
import pydeck as pdk
import geopandas as gpda
import config
import json
import requests
import sys
import folium

path = "C:\github/aide-analysis/"
sys.path.append(path)

from src.sql import DatabaseHandler
db = DatabaseHandler()

from core.config import Config
config = Config()


In [ ]:
# Vworld 시군구 경계 API 호출
def gl(sig_cd):
    # 엔드포인트
    endpoint = "http://api.vworld.kr/req/data"

    # API 파라미터
    service = "data"
    request = "GetFeature"

    # 행정동 경계
    data = "LT_C_ADSIGG_INFO"

    # 요청 파라미터
    key = config.OPEN_API["vworld"]
    page = 1
    size = 1000
    
    attrFilter = f"sig_cd:like:{sig_cd}"
    url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"

    # 요청 결과
    res = json.loads(requests.get(url).text)

    # GeoJson 생성
    featureCollection = {"type": "FeatureCollection"}
    features = []
    while True:
        # 요청 URL
        url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"
        # 요청 결과
        res = json.loads(requests.get(url).text)
        # GeoJson 추가
        features = features + res["response"]["result"]["featureCollection"]["features"]
        # 페이지
        page_current = int(res["response"]["page"]["current"])
        page_total = int(res["response"]["page"]["total"])
        page += 1
        if page_current == page_total:
            break
    featureCollection["features"] = features
    geo = featureCollection
    return geo
geo = gl(11)

In [ ]:
# params
center = [37.541, 126.986]
lon = 127.0080061098534
lat = 37.50989980338221
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']


# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    width = 750,
    height = 500,
    tiles = tiles[2]
)


# Marker
"""
location : 마커를 추가할 경위도 좌표/ 
popup : 표기할 팝업문구(클릭시 문구)/ 
tooltip : 표기할 툴팁 지정(마우스 오버))
"""
folium.Marker(
    location = [lat, lon],
    popup = '반포자이',
    tooltip = '반포자이',
    icon = folium.Icon('red', icon = icon[0])
).add_to(m)

# GeoJson 활용
folium.GeoJson(
    geo,
    name = '서초구'
).add_to(m)

m
# m.save('data/result.html')

In [ ]:
# m.save('result/서울시군구경계시각화.html')

### DB연결후 적용

#### 세대수

In [ ]:
# 데이터 불러오기
sql = f"""
SELECT hdongName, 
       totalSaedae 
FROM aide.population_saedae
WHERE sidoCode ='11' 
      and sigunguCode != '11000' 
      and date = (SELECT MAX(date) 
                  FROM aide.population_saedae);
"""
df = db.read_table(sql)

In [ ]:

# params
center = [37.541, 126.986]
lon = 127.0080061098534
lat = 37.50989980338221
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']


# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    tiles = tiles[2]
)

# Marker
folium.Marker(
    location = [37.5146321, 127.018293],
    popup = '회사',
    tooltip = '회사',
    icon = folium.Icon('red', icon = icon[0])
).add_to(m)


# Marker
folium.Marker(
    location = [37.4857983, 126.948162],
    popup = '집',
    tooltip = '집',
    icon = folium.Icon('red', icon = icon[0])
).add_to(m)


# Polygon
m.choropleth(
    geo_data = geo,
    data = df,
    columns = ('hdongName','totalSaedae'),
    key_on = 'feature.properties.full_nm',
    fill_color = 'YlOrRd',
    fill_opacity = 0.5,
    line_opacity = 0.2
)
m

In [ ]:
# m.save('result/서울시시군구세대수시각화.html')

In [ ]:
location_data = [[37.5146321, 127.018293],
                [37.4857983, 126.948162]]
folium.PolyLine(locations = location_data, tooltip = "Polyline").add_to(m)
m

### 연속지적도 + 아파트 전세 공급평단가(최근날짜기준)

In [18]:
sql = f"""
SELECT T1.pnuCode,
	   T1.contractYear,
       T1.contractMonth,
       T1.tradingSupplyUnitPriceAverage,
       T1.jeonseSupplyUnitPriceAverage,
       T1.wolseSupplyUnitPriceAverage,
       T1.rn
FROM(
SELECT pnuCode, 
	   contractYear, 
	   contractMonth,
       MAX(tradingSupplyUnitPriceAverage) as tradingSupplyUnitPriceAverage,
       MAX(jeonseSupplyUnitPriceAverage) AS jeonseSupplyUnitPriceAverage,
       MAX(wolseSupplyUnitPriceAverage) AS wolseSupplyUnitPriceAverage,
       row_number() OVER(PARTITION BY pnuCode ORDER BY contractYear DESC) AS rn
       
FROM aide.apartment_danji_monthly_price
WHERE pnuCode LIKE '41463%'
GROUP BY pnuCode, contractYear, contractMonth 
) AS T1
WHERE T1.rn=  1
"""
price_df = db.read_table(sql)

In [19]:
price_df = price_df.dropna(subset = 'jeonseSupplyUnitPriceAverage')
price_df = price_df[['pnuCode','contractYear','contractMonth','jeonseSupplyUnitPriceAverage']]
price_df

,pnuCode,contractYear,contractMonth,jeonseSupplyUnitPriceAverage
0,4146310100100020001,2022,1,1308.515162
1,4146310100100060009,2022,3,565.813037
2,4146310100100140000,2022,1,915.155935
3,4146310100100140004,2022,2,1091.504994
4,4146310100100170015,2022,1,1142.190593
...,...,...,...,...
259,4146311800112910000,2022,2,1200.735724
260,4146311800112930000,2022,1,973.119756
261,4146311800112980000,2022,1,913.683445
262,4146311800113930000,2022,1,1043.598671


In [20]:
# join
lst = ",".join(price_df['pnuCode'])

In [21]:
def continue_land():
    # 엔드포인트
    endpoint = "http://api.vworld.kr/req/data"

    # API 파라미터
    service = "data"
    request = "GetFeature"

    # 연속지적도
    data = "LP_PA_CBND_BUBUN"

    # 요청 파라미터
    key = config.OPEN_API["vworld"]
    page = 1
    size = 1000
    
    attrFilter = f'pnu:IN:{lst}'
    url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"

    # 요청 결과
    res = json.loads(requests.get(url).text)

    # GeoJson 생성
    featureCollection = {"type": "FeatureCollection"}
    features = []
    while True:
        # 요청 URL
        url = f"{endpoint}?service={service}&request={request}&data={data}&key={key}&attrFilter={attrFilter}&page={page}&size={size}"
        # 요청 결과
        res = json.loads(requests.get(url).text)
        # GeoJson 추가
        features = features + res["response"]["result"]["featureCollection"]["features"]
        # 페이지
        page_current = int(res["response"]["page"]["current"])
        page_total = int(res["response"]["page"]["total"])
        page += 1
        if page_current== page_total:
            break 
    featureCollection["features"] = features
    geo = featureCollection
    return geo

geo = continue_land()

In [ ]:
geo

In [22]:

# params
center = [37.541, 126.986]
lon = 127.0080061098534
lat = 37.50989980338221
icon = ['star','flag']
tiles = ['cartodbpositron', 'Stamen Toner', 'OpenStreetMap']


# Base Map
m = folium.Map(
    location = [center[0], center[1]],
    zoom_start = 12,
    tiles = tiles[0]
)


# Polygon
m.choropleth(
    geo_data = geo,
    data = price_df,
    columns = ('pnuCode','jeonseSupplyUnitPriceAverage'),
    key_on = 'feature.properties.pnu',
    fill_color = 'YlOrRd',
    fill_opacity = 0.5,
    line_opacity = 0.2
)
m

C:\Users\jeongseokLim\anaconda3\envs\aide\lib\site-packages\folium\folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [ ]:
m.save('result/testfridat.html')